In [1]:
import os
import torch
import pwcca
import random
import librosa
import opensmile
import numpy as np
import pandas as pd
import torch.nn as nn
import soundfile as sf
from scipy import signal
from sklearn.preprocessing import StandardScaler, normalize
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
w2v_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base").to(device)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/afs/inf.ed.ac.uk/user/s20/s2057508/.local/lib/python3.6/site-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_hid.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

In [3]:
# calculate correlation

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)


path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/IEMOCAP/Data'
os.chdir(path)

# hct_frame = np.array([])
# hct_phone = np.array([])
hct_word = np.array([])
# hct_utter = np.array([])
w2v0_feats = np.array([])
w2v1_feats = np.array([])
w2v2_feats = np.array([])
w2v3_feats = np.array([])
w2v4_feats = np.array([])
w2v5_feats = np.array([])
w2v6_feats = np.array([])
w2v7_feats = np.array([])
w2v8_feats = np.array([])
w2v9_feats = np.array([])
w2v10_feats = np.array([])
w2v11_feats = np.array([])
w2v12_feats = np.array([])

with open('emo_name_trans_5500.txt', 'r') as f:
    files = f.readlines()
    files = np.array(files)
    indices = np.arange(len(files))
    random.shuffle(indices)
    files[np.arange(len(files))] = files[indices]
    n = 1
    for line in files:
        print(n)
        emotion = line.split(" ")[0]
        file_name = line.split(" ")[1]
        audioinput, sr = librosa.load(file_name, sr=16000)
        if len(audioinput) > 100000:
            continue
#         audioinput = librosa.util.fix_length(audioinput, size=120000)

        torch.cuda.empty_cache()

        egmaps = smile.process_signal(audioinput, sr).values
#         print(egmaps.shape)
    
        #w2v features
        input_values = processor(audioinput, sampling_rate=16000, return_tensors="pt").input_values.to(device)
        w2v_feats = w2v_model(input_values, output_hidden_states=True).hidden_states
        w2v0 = np.array(w2v_feats[0].detach()[0].cpu())
        w2v1 = np.array(w2v_feats[1].detach()[0].cpu())
        w2v2 = np.array(w2v_feats[2].detach()[0].cpu())
        w2v3 = np.array(w2v_feats[3].detach()[0].cpu())
        w2v4 = np.array(w2v_feats[4].detach()[0].cpu())
        w2v5 = np.array(w2v_feats[5].detach()[0].cpu())
        w2v6 = np.array(w2v_feats[6].detach()[0].cpu())
        w2v7 = np.array(w2v_feats[7].detach()[0].cpu())
        w2v8 = np.array(w2v_feats[8].detach()[0].cpu())
        w2v9 = np.array(w2v_feats[9].detach()[0].cpu())
        w2v10 = np.array(w2v_feats[10].detach()[0].cpu())
        w2v11 = np.array(w2v_feats[11].detach()[0].cpu())
        w2v12 = np.array(w2v_feats[12].detach()[0].cpu())
        
#         egmaps_frame = signal.resample(egmaps, len(w2v0))
        
        egmaps_phone = []
        for i in range(0,len(egmaps)-3, 3):
            egmaps_phone.append((egmaps[i]+egmaps[i+1]+egmaps[i+2])/3.0) #phoneme level   
        egmaps_word=[]
        for i in range(0, len(egmaps_phone)-5, 5):
            egmaps_word.append((egmaps_phone[i]+egmaps_phone[i+1]+egmaps_phone[i+2]+egmaps_phone[i+3]+egmaps_phone[i+4])/5.0) #word level
#         egmaps_utter = np.mean(egmaps, axis=0).reshape(1, -1)
#         print(egmaps_utter.shape)
#         print(w2v0.shape)
        w2v0 = signal.resample(w2v0, len(egmaps_word))
        w2v1 = signal.resample(w2v1, len(egmaps_word))
        w2v2 = signal.resample(w2v2, len(egmaps_word))
        w2v3 = signal.resample(w2v3, len(egmaps_word))
        w2v4 = signal.resample(w2v4, len(egmaps_word))
        w2v5 = signal.resample(w2v5, len(egmaps_word))
        w2v6 = signal.resample(w2v6, len(egmaps_word))
        w2v7 = signal.resample(w2v7, len(egmaps_word))
        w2v8 = signal.resample(w2v8, len(egmaps_word))
        w2v9 = signal.resample(w2v9, len(egmaps_word))
        w2v10 = signal.resample(w2v10, len(egmaps_word))
        w2v11 = signal.resample(w2v11, len(egmaps_word))
        w2v12 = signal.resample(w2v12, len(egmaps_word))

#for utterance level
#         w2v0 = np.mean(signal.resample(w2v0, 2), axis=0).reshape(1, -1)
#         w2v1 = np.mean(signal.resample(w2v1, 2), axis=0).reshape(1, -1)
#         w2v2 = np.mean(signal.resample(w2v2, 2), axis=0).reshape(1, -1)
#         w2v3 = np.mean(signal.resample(w2v3, 2), axis=0).reshape(1, -1)
#         w2v4 = np.mean(signal.resample(w2v4, 2), axis=0).reshape(1, -1)
#         w2v5 = np.mean(signal.resample(w2v5, 2), axis=0).reshape(1, -1)
#         w2v6 = np.mean(signal.resample(w2v6, 2), axis=0).reshape(1, -1)
#         w2v7 = np.mean(signal.resample(w2v7, 2), axis=0).reshape(1, -1)
#         w2v8 = np.mean(signal.resample(w2v8, 2), axis=0).reshape(1, -1)
#         w2v9 = np.mean(signal.resample(w2v9, 2), axis=0).reshape(1, -1)
#         w2v10 = np.mean(signal.resample(w2v10, 2), axis=0).reshape(1, -1)
#         w2v11 = np.mean(signal.resample(w2v11, 2), axis=0).reshape(1, -1)
#         w2v12 = np.mean(signal.resample(w2v12, 2), axis=0).reshape(1, -1)
        
#         print(w2v1.shape, egmaps_utter.shape)
#         print(w2v1_feats.shape)

        torch.cuda.empty_cache()
        
        if len(hct_word) > 0:
            w2v0_feats = np.concatenate((w2v0_feats, w2v0), axis=0)
            w2v1_feats = np.concatenate((w2v1_feats, w2v1), axis=0)
            w2v2_feats = np.concatenate((w2v2_feats, w2v2), axis=0)
            w2v3_feats = np.concatenate((w2v3_feats, w2v3), axis=0)
            w2v4_feats = np.concatenate((w2v4_feats, w2v4), axis=0)
            w2v5_feats = np.concatenate((w2v5_feats, w2v5), axis=0)
            w2v6_feats = np.concatenate((w2v6_feats, w2v6), axis=0)
            w2v7_feats = np.concatenate((w2v7_feats, w2v7), axis=0)
            w2v8_feats = np.concatenate((w2v8_feats, w2v8), axis=0)
            w2v9_feats = np.concatenate((w2v9_feats, w2v9), axis=0)
            w2v10_feats = np.concatenate((w2v10_feats, w2v10), axis=0)
            w2v11_feats = np.concatenate((w2v11_feats, w2v11), axis=0)
            w2v12_feats = np.concatenate((w2v12_feats, w2v12), axis=0)
            hct_word = np.concatenate((hct_word, egmaps_word), axis=0)
        else:
            w2v0_feats = w2v0
            w2v1_feats = w2v1
            w2v2_feats = w2v2
            w2v3_feats = w2v3
            w2v4_feats = w2v4
            w2v5_feats = w2v5
            w2v6_feats = w2v6
            w2v7_feats = w2v7
            w2v8_feats = w2v8
            w2v9_feats = w2v9
            w2v10_feats = w2v10 
            w2v11_feats = w2v11
            w2v12_feats = w2v12
            hct_word = egmaps_word
            
        n += 1
        if n == 4295:
            break

1
2
2
3
4
5
6
7
8
8
9
10
10
11
12
13
14
15
16
17
18
19
20
21
22
23
23
24
25
26
26
27
28
29
30
31
32
33
33
34
35
36
36
37
37
37
37
38
39
40
41
42
43
44
45
46
47
48
49
49
50
51
52
53
53
54
55
56
56
57
58
59
60
61
62
63
64
65
66
67
68
68
69
70
71
72
73
74
75
76
76
76
77
78
79
80
81
82
83
84
85
85
86
86
87
88
89
90
90
91
91
92
93
93
94
95
96
97
97
97
98
99
100
101
101
102
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
116
117
117
118
119
119
119
120
121
121
122
123
124
124
125
126
127
128
129
130
131
132
133
134
134
135
136
137
137
137
138
139
140
141
142
143
144
145
146
147
147
148
148
149
150
150
151
152
153
153
154
155
156
156
157
158
159
160
161
162
162
163
164
165
166
167
167
168
169
170
171
172
173
174
175
175
175
176
177
177
178
179
180
181
181
182
183
183
184
185
186
187
188
188
188
189
189
190
191
192
193
194
194
195
196
197
197
198
199
200
201
202
203
204
205
206
206
207
208
209
210
211
212
213
214
215
216
217
217
217
217
218
218
218
219
220
221
222
222
223
224
225
2

1500
1501
1502
1502
1503
1504
1505
1506
1507
1508
1508
1509
1510
1510
1511
1512
1513
1514
1515
1515
1516
1517
1518
1519
1520
1521
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1535
1536
1537
1538
1539
1540
1541
1542
1542
1543
1543
1543
1544
1545
1545
1546
1547
1548
1549
1549
1549
1550
1550
1550
1551
1551
1551
1552
1553
1553
1554
1554
1554
1555
1556
1557
1558
1559
1559
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1569
1569
1569
1570
1571
1572
1573
1573
1573
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1588
1588
1588
1588
1589
1590
1590
1590
1591
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1601
1602
1603
1603
1604
1605
1606
1607
1608
1609
1609
1610
1611
1612
1613
1614
1615
1616
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1636
1637
1637
1637
1638
1638
1639
1639
1640
1641
1642
1643
1644
1645
1646
1646
1647
1648
1648
1649
1650
1651
1652
1653
1653


2783
2784
2784
2784
2785
2786
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2803
2804
2805
2805
2806
2807
2808
2809
2810
2811
2812
2812
2813
2814
2815
2816
2817
2818
2818
2819
2819
2819
2820
2820
2820
2821
2822
2823
2824
2825
2826
2826
2827
2827
2828
2829
2829
2829
2830
2830
2831
2832
2833
2834
2835
2836
2837
2838
2838
2839
2840
2841
2841
2842
2843
2844
2845
2846
2846
2847
2848
2849
2849
2850
2851
2851
2851
2852
2853
2853
2853
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2863
2864
2864
2864
2865
2866
2867
2867
2867
2868
2869
2870
2871
2872
2873
2873
2873
2874
2875
2876
2877
2878
2878
2879
2880
2881
2881
2882
2883
2884
2884
2885
2886
2887
2887
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2897
2898
2899
2900
2901
2902
2902
2902
2903
2904
2905
2905
2905
2906
2907
2907
2907
2907
2908
2908
2909
2910
2911
2911
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934


4065
4066
4067
4067
4067
4068
4069
4069
4070
4071
4072
4073
4074
4075
4076
4076
4077
4078
4079
4080
4081
4081
4081
4082
4083
4084
4085
4086
4087
4088
4088
4089
4089
4090
4091
4092
4092
4093
4094
4095
4095
4096
4097
4098
4099
4100
4101
4101
4102
4102
4102
4102
4103
4103
4104
4105
4105
4106
4107
4108
4109
4109
4110
4111
4112
4113
4114
4115
4115
4116
4116
4117
4118
4119
4120
4121
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4137
4138
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4155
4156
4157
4158
4159
4160
4161
4162
4162
4163
4163
4164
4165
4166
4166
4167
4168
4168
4168
4169
4170
4171
4172
4172
4172
4173
4174
4175
4176
4177
4177
4178
4179
4180
4181
4182
4182
4182
4183
4183
4184
4185
4186
4187
4187
4188
4189
4190
4191
4191
4192
4193
4193
4194
4195
4195
4195
4196
4197
4197
4198
4199
4199
4200
4201
4202
4203
4204
4205
4206
4207
4207
4208
4208
4209
4210
4211
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220


In [4]:
print(hct_word.shape, w2v1_feats.shape)

(87036, 25) (87036, 768)


In [5]:
corr_frame0, _, coefs0 = pwcca.compute_pwcca(hct_word.T, w2v0_feats.T, epsilon=1e-10)
corr_frame1, _, coefs1 = pwcca.compute_pwcca(hct_word.T, w2v1_feats.T, epsilon=1e-10)
corr_frame2, _, coefs2 = pwcca.compute_pwcca(hct_word.T, w2v2_feats.T, epsilon=1e-10)
corr_frame3, _, coefs3 = pwcca.compute_pwcca(hct_word.T, w2v3_feats.T, epsilon=1e-10)
corr_frame4, _, coefs4 = pwcca.compute_pwcca(hct_word.T, w2v4_feats.T, epsilon=1e-10)
corr_frame5, _, coefs5 = pwcca.compute_pwcca(hct_word.T, w2v5_feats.T, epsilon=1e-10)
corr_frame6, _, coefs6 = pwcca.compute_pwcca(hct_word.T, w2v6_feats.T, epsilon=1e-10)
corr_frame7, _, coefs7 = pwcca.compute_pwcca(hct_word.T, w2v7_feats.T, epsilon=1e-10)
corr_frame8, _, coefs8 = pwcca.compute_pwcca(hct_word.T, w2v8_feats.T, epsilon=1e-10)
corr_frame9, _, coefs9 = pwcca.compute_pwcca(hct_word.T, w2v9_feats.T, epsilon=1e-10)
corr_frame10, _, coefs10 = pwcca.compute_pwcca(hct_word.T, w2v10_feats.T, epsilon=1e-10)
corr_frame11, _, coefs11 = pwcca.compute_pwcca(hct_word.T, w2v11_feats.T, epsilon=1e-10)
corr_frame12, _, coefs12 = pwcca.compute_pwcca(hct_word.T, w2v12_feats.T, epsilon=1e-10)

print('Layer 0')
# print(corr_frame0)
print(np.mean(coefs0))

print('Layer 1')
# print(corr_frame1)
print(np.mean(coefs1))

print('Layer 2')
# print(corr_frame2)
print(np.mean(coefs2))

print('Layer 3')
# print(corr_frame3)
print(np.mean(coefs3))

print('Layer 4')
# print(corr_frame4)
print(np.mean(coefs4))

print('Layer 5')
# print(corr_frame5)
print(np.mean(coefs5))

print('Layer 6')
# print(corr_frame6)
print(np.mean(coefs6))

print('Layer 7')
# print(corr_frame7)
print(np.mean(coefs7))

print('Layer 8')
# print(corr_frame8)
print(np.mean(coefs8))

print('Layer 9')
# print(corr_frame9)
print(np.mean(coefs9))

print('Layer 10')
# print(corr_frame10)
print(np.mean(coefs10))

print('Layer 11')
# print(corr_frame11)
print(np.mean(coefs11))

print('Layer 12')
# print(corr_frame12)
print(np.mean(coefs12))

Layer 0
0.5974304200997829
Layer 1
0.6145489794741078
Layer 2
0.6065558647616632
Layer 3
0.5812102024743842
Layer 4
0.5484689485182761
Layer 5
0.515029720968782
Layer 6
0.4877074024471515
Layer 7
0.4737600938798658
Layer 8
0.47364356216402087
Layer 9
0.5082643437132259
Layer 10
0.5472375227737571
Layer 11
0.45096036454273053
Layer 12
0.49999500646071093


In [2]:
#phoneme level - frame level  PT model
print( 0.5709333219363342  -  0.5393215027762929 )
print( 0.5736474305246732  -  0.5406392619324453 )
print( 0.5631062087329875  -  0.5361511743311935 )
print( 0.5364144394879908  -  0.5036942069772349 )
print( 0.5024256786336995  -  0.4709580684454375 )
print( 0.4677339765936467  -  0.4363481313538615 )
print( 0.4396573414143345  -  0.4126715162231631 )
print( 0.4306357694463532  -  0.3997789653139901 )
print( 0.4307230486483944  -  0.4010562800118452 )
print( 0.4612588853947390  -  0.4285008845583443 )
print( 0.5004507796408333  -  0.4620553998984243 )
print( 0.4067616078978662  -  0.3452946501361398 )
print( 0.4703225438808748  -  0.4205991749365979 )

0.0316118191600413
0.03300816859222788
0.02695503440179392
0.03272023251075595
0.03146761018826194
0.03138584523978516
0.026985825191171375
0.03085680413236308
0.029666768636549212
0.03275800083639474
0.038395379742408975
0.06146695776172639
0.049723368944276924


In [3]:
#word level - phoneme level  PT model
print( 0.597430420099782  -  0.5709333219363342 )
print( 0.614548979474107  -  0.5736474305246732 )
print( 0.606555864761663  -  0.5631062087329875 )
print( 0.581210202474384  -  0.5364144394879908 )
print( 0.548468948518276  -  0.5024256786336995 )
print( 0.515029720968782  -  0.4677339765936467 )
print( 0.487707402447151  -  0.4396573414143345 )
print( 0.473760093879865  -  0.4306357694463532 )
print( 0.473643562164020  -  0.4307230486483944 )
print( 0.508264343713225  -  0.4612588853947390 )
print( 0.547237522773757  -  0.5004507796408333 )
print( 0.450960364542730  -  0.4067616078978662 )
print( 0.499995006460710  -  0.4703225438808748 )

0.026497098163447808
0.040901548949433875
0.043449656028675565
0.044795762986393206
0.04604326988457652
0.047295744375135285
0.048050061032816505
0.04312432443351183
0.04292051351562559
0.047005458318486015
0.04678674313292375
0.04419875664486378
0.02967246257983519


In [5]:
#phoneme level - frame level  FT960 model

print( 0.5679434323619642  -  0.530143005502070 )
print( 0.5766334730924673  -  0.532767823782619 )
print( 0.5659710620873298  -  0.525615747930049 )
print( 0.5375288991855908  -  0.500326919099980 )
print( 0.5007393645398366  -  0.467158076569302 )
print( 0.4642629908288020  -  0.435640083230318 )
print( 0.4350782299757341  -  0.408367106228152 )
print( 0.4110680986432208  -  0.386043688512377 )
print( 0.3944439492160639  -  0.370069979326406 )
print( 0.3820931212708214  -  0.361198780987203 )
print( 0.3604261198988777  -  0.339451021271471 )
print( 0.3266731104980963  -  0.303901823296069 )
print( 0.3063574241936074  -  0.281612891797627 )

0.037800426859894154
0.043865649309848265
0.040355314157280886
0.03720198008561082
0.03358128797053456
0.028622907598484015
0.02671112374758211
0.025024410130843777
0.024373969889657898
0.020894340283618418
0.020975098627406707
0.022771287202027257
0.0247445323959804


In [6]:
#word level - phoneme level  FT960 model

print( 0.619356523707548  -  0.5679434323619642 )
print( 0.624696739371678  -  0.5766334730924673 )
print( 0.613054149675998  -  0.5659710620873298 )
print( 0.585729805371353  -  0.5375288991855908 )
print( 0.548931851610061  -  0.5007393645398366 )
print( 0.513186087149223  -  0.4642629908288020 )
print( 0.483912690978468  -  0.4350782299757341 )
print( 0.460011056531492  -  0.4110680986432208 )
print( 0.443438189266197  -  0.3944439492160639 )
print( 0.433631788249363  -  0.3820931212708214 )
print( 0.417541246304359  -  0.3604261198988777 )
print( 0.391887091422830  -  0.3266731104980963 )
print( 0.363708292023395  -  0.3063574241936074 )

0.051413091345583806
0.04806326627921076
0.04708308758866819
0.04820090618576223
0.04819248707022439
0.04892309632042097
0.04883446100273392
0.04894295788827119
0.04899424005013309
0.05153866697854159
0.05711512640548133
0.06521398092473374
0.0573508678297876
